# Analisis Sinyal Nonstasioner
Nehemy Davis Suryanto

5023221004

---

## Load Data

In [48]:
# Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objs as go

# Load txt file
df = pd.read_csv('data/ppg_signal_wean.txt', sep='\t')
df.columns = [col.lower() for col in df.columns]
signal = df['red'] if 'red' in df.columns else df.iloc[:,0]

sampling_rate = 200
time = np.arange(len(signal)) / sampling_rate
total_time = round(time[-1])

# Interactive plot with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=time, y=signal, mode='lines', name='PPG Signal', line=dict(color='crimson')))
fig.update_layout(title='PPG Signal (Red Channel)',
                  xaxis_title='Time (seconds)',
                  yaxis_title='Amplitude',
                  template='plotly_white',
                  font=dict(size=14),
                  margin=dict(l=40, r=40, t=60, b=40))
fig.show()

## Preprocessing

In [49]:
from scipy.signal import resample

# Calculate new number of samples
target_sampling_rate = 50
num_samples = int(len(signal) * target_sampling_rate / sampling_rate)
signal_ds = resample(signal, num_samples)
time_ds = np.arange(len(signal_ds)) / target_sampling_rate

print(f"Original number of samples: {len(signal)}")
print(f"Downsampled number of samples: {len(signal_ds)}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_ds, y=signal_ds, mode='lines', name='PPG Signal (Downsampled)', line=dict(color='royalblue')))
fig.update_layout(title='PPG Signal (Downsampled to 50 Hz)',
                  xaxis_title='Time (seconds)',
                  yaxis_title='Amplitude',
                  template='plotly_white',
                  font=dict(size=14),
                  margin=dict(l=40, r=40, t=60, b=40))
fig.show()

Original number of samples: 12000
Downsampled number of samples: 3000


## Discrete Wavelet Transform

In [50]:
def dirac(x):
    dirac_delta = 1 if (x == 0) else 0
    return dirac_delta

h = []
g = []
n_list = []

for n in range(-2, 2):
    n_list.append(n)
    temp_h = 1/8 * (dirac(n-1) + 3*dirac(n) + 3*dirac(n+1) + dirac(n+2))
    h.append(temp_h)
    temp_g = -2 * (dirac(n) - dirac(n+1))
    g.append(temp_g)

print('h(n)')
fig = go.Figure()
fig.add_trace(go.Bar(x=n_list, y=h, name='h(n)'))
fig.update_layout(title='h(n)', xaxis_title='n', yaxis_title='h(n)', template='plotly_white')
fig.show()

print('g(n)')
fig = go.Figure()
fig.add_trace(go.Bar(x=n_list, y=g, name='g(n)'))
fig.update_layout(title='g(n)', xaxis_title='n', yaxis_title='g(n)', template='plotly_white')
fig.show()

qj = np.zeros((9, 100000))

k_list = []
j = 1
print('Scale =', j)
a = -(round(2**j) + round(2**(j-1)) - 2)
print('a =', a)
b = -(1 - round(2**(j-1))) + 1
print('b =', b)

for k in range(a, b):
    k_list.append(k)
    qj[1][k + abs(a)] = -2 * (dirac(k) - dirac(k+1))

fig = go.Figure()
fig.add_trace(go.Bar(x=k_list, y=qj[1][0:len(k_list)], name=f'q{j}(k)'))
fig.update_layout(title=f'Scale {j}', xaxis_title='k', yaxis_title='q(k)', template='plotly_white')
fig.show()

k_list = []
j = 2
print('Scale =', j)
a = -(round(2**j) + round(2**(j-1)) - 2)
print('a =', a)
b = -(1 - round(2**(j-1))) + 1
print('b =', b)

for k in range(a, b):
    k_list.append(k)
    qj[2][k + abs(a)] = -1/4 * (dirac(k-1) + 3*dirac(k) + 2*dirac(k+1) 
                                - 2*dirac(k+2) - 3*dirac(k+3) - dirac(k+4))

fig = go.Figure()
fig.add_trace(go.Bar(x=k_list, y=qj[2][0:len(k_list)], name=f'q{j}(k)'))
fig.update_layout(title=f'Scale {j}', xaxis_title='k', yaxis_title='q(k)', template='plotly_white')
fig.show()

k_list = []
j = 3
print('Scale =', j)
a = -(round(2**j) + round(2**(j-1)) - 2)
print('a =', a)
b = -(1 - round(2**(j-1))) + 1
print('b =', b)

for k in range(a, b):
    k_list.append(k)
    qj[3][k + abs(a)] = -1/32 * (
        dirac(k-3) + 3*dirac(k-2) + 6*dirac(k-1) + 10*dirac(k) + 11*dirac(k+1) +
        9*dirac(k+2) + 4*dirac(k+3) - 4*dirac(k+4) - 9*dirac(k+5) - 11*dirac(k+6) -
        10*dirac(k+7) - 6*dirac(k+8) - 3*dirac(k+9) - dirac(k+10)
    )

fig = go.Figure()
fig.add_trace(go.Bar(x=k_list, y=qj[3][0:len(k_list)], name=f'q{j}(k)'))
fig.update_layout(title=f'Scale {j}', xaxis_title='k', yaxis_title='q(k)', template='plotly_white')
fig.show()

k_list = []
j = 4
print('Scale =', j)
a = -(round(2**j) + round(2**(j-1)) - 2)
print('a =', a)
b = -(1 - round(2**(j-1))) + 1
print('b =', b)

for k in range(a, b):
    k_list.append(k)
    qj[4][k + abs(a)] = -1/256 * (
        dirac(k-7) + 3*dirac(k-6) + 6*dirac(k-5) + 10*dirac(k-4) + 15*dirac(k-3) +
        21*dirac(k-2) + 28*dirac(k-1) + 36*dirac(k) + 41*dirac(k+1) + 43*dirac(k+2) +
        42*dirac(k+3) + 38*dirac(k+4) + 31*dirac(k+5) + 21*dirac(k+6) + 8*dirac(k+7) -
        8*dirac(k+8) - 21*dirac(k+9) - 31*dirac(k+10) - 38*dirac(k+11) - 42*dirac(k+12) -
        43*dirac(k+13) - 41*dirac(k+14) - 36*dirac(k+15) - 28*dirac(k+16) -
        21*dirac(k+17) - 15*dirac(k+18) - 10*dirac(k+19) - 6*dirac(k+20) -
        3*dirac(k+21) - dirac(k+22)
    )

fig = go.Figure()
fig.add_trace(go.Bar(x=k_list, y=qj[4][0:len(k_list)], name=f'q{j}(k)'))
fig.update_layout(title=f'Scale {j}', xaxis_title='k', yaxis_title='q(k)', template='plotly_white')
fig.show()

k_list = []
j = 5
print('Scale =', j)
a = -(round(2**j) + round(2**(j-1)) - 2)
print('a =', a)
b = -(1 - round(2**(j-1))) + 1
print('b =', b)

for k in range(a, b):
    k_list.append(k)
    qj[5][k + abs(a)] = -1/512 * (
        dirac(k-15) + 3*dirac(k-14) + 6*dirac(k-13) + 10*dirac(k-12) + 15*dirac(k-11) +
        21*dirac(k-10) + 28*dirac(k-9) + 36*dirac(k-8) + 45*dirac(k-7) + 55*dirac(k-6) +
        66*dirac(k-5) + 78*dirac(k-4) + 91*dirac(k-3) + 105*dirac(k-2) + 120*dirac(k-1) +
        136*dirac(k) + 149*dirac(k+1) + 159*dirac(k+2) + 166*dirac(k+3) + 170*dirac(k+4) +
        171*dirac(k+5) + 169*dirac(k+6) + 164*dirac(k+7) + 156*dirac(k+8) + 145*dirac(k+9) +
        131*dirac(k+10) + 114*dirac(k+11) + 94*dirac(k+12) + 71*dirac(k+13) + 45*dirac(k+14) +
        16*dirac(k+15) - 16*dirac(k+16) - 45*dirac(k+17) - 71*dirac(k+18) - 94*dirac(k+19) -
        114*dirac(k+20) - 131*dirac(k+21) - 145*dirac(k+22) - 156*dirac(k+23) -
        164*dirac(k+24) - 169*dirac(k+25) - 171*dirac(k+26) - 170*dirac(k+27) -
        166*dirac(k+28) - 159*dirac(k+29) - 149*dirac(k+30) - 136*dirac(k+31) -
        120*dirac(k+32) - 105*dirac(k+33) - 91*dirac(k+34) - 78*dirac(k+35) -
        66*dirac(k+36) - 55*dirac(k+37) - 45*dirac(k+38) - 36*dirac(k+39) -
        28*dirac(k+40) - 21*dirac(k+41) - 15*dirac(k+42) - 10*dirac(k+43) -
        6*dirac(k+44) - 3*dirac(k+45) - dirac(k+46)
    )

fig = go.Figure()
fig.add_trace(go.Bar(x=k_list, y=qj[5][0:len(k_list)], name=f'q{j}(k)'))
fig.update_layout(title=f'Scale {j}', xaxis_title='k', yaxis_title='q(k)', template='plotly_white')
fig.show()

k_list = []
j = 6
print('Scale =', j)
a = -(round(2**j) + round(2**(j-1)) - 2)  
b = -(1 - round(2**(j-1))) + 1           
for k in range(a, b):
    k_list.append(k)
    qj[6][k + abs(a)] = -1 / 2048 * (
        dirac(k - 31) + 3 * dirac(k - 30) + 6 * dirac(k - 29) + 10 * dirac(k - 28) +
        15 * dirac(k - 27) + 21 * dirac(k - 26) + 28 * dirac(k - 25) + 36 * dirac(k - 24) +
        45 * dirac(k - 23) + 55 * dirac(k - 22) + 66 * dirac(k - 21) + 78 * dirac(k - 20) +
        91 * dirac(k - 19) + 105 * dirac(k - 18) + 120 * dirac(k - 17) + 136 * dirac(k - 16) +
        153 * dirac(k - 15) + 171 * dirac(k - 14) + 190 * dirac(k - 13) + 210 * dirac(k - 12) +
        231 * dirac(k - 11) + 253 * dirac(k - 10) + 276 * dirac(k - 9) + 300 * dirac(k - 8) +
        325 * dirac(k - 7) + 351 * dirac(k - 6) + 378 * dirac(k - 5) + 406 * dirac(k - 4) +
        435 * dirac(k - 3) + 465 * dirac(k - 2) + 496 * dirac(k - 1) + 528 * dirac(k) +
        557 * dirac(k + 1) + 583 * dirac(k + 2) + 606 * dirac(k + 3) + 626 * dirac(k + 4) +
        643 * dirac(k + 5) + 657 * dirac(k + 6) + 668 * dirac(k + 7) + 676 * dirac(k + 8) +
        681 * dirac(k + 9) + 683 * dirac(k + 10) + 682 * dirac(k + 11) + 678 * dirac(k + 12) +
        671 * dirac(k + 13) + 661 * dirac(k + 14) + 648 * dirac(k + 15) + 632 * dirac(k + 16) +
        613 * dirac(k + 17) + 591 * dirac(k + 18) + 566 * dirac(k + 19) + 538 * dirac(k + 20) +
        507 * dirac(k + 21) + 473 * dirac(k + 22) + 436 * dirac(k + 23) + 396 * dirac(k + 24) +
        353 * dirac(k + 25) + 307 * dirac(k + 26) + 258 * dirac(k + 27) + 206 * dirac(k + 28) +
        151 * dirac(k + 29) + 93 * dirac(k + 30) + 32 * dirac(k + 31) - 32 * dirac(k + 32) -
        93 * dirac(k + 33) - 151 * dirac(k + 34) - 206 * dirac(k + 35) - 258 * dirac(k + 36) -
        307 * dirac(k + 37) - 353 * dirac(k + 38) - 396 * dirac(k + 39) - 436 * dirac(k + 40) -
        473 * dirac(k + 41) - 507 * dirac(k + 42) - 538 * dirac(k + 43) - 566 * dirac(k + 44) -
        591 * dirac(k + 45) - 613 * dirac(k + 46) - 632 * dirac(k + 47) - 648 * dirac(k + 48) -
        661 * dirac(k + 49) - 671 * dirac(k + 50) - 678 * dirac(k + 51) - 682 * dirac(k + 52) -
        683 * dirac(k + 53) - 681 * dirac(k + 54) - 676 * dirac(k + 55) - 668 * dirac(k + 56) -
        657 * dirac(k + 57) - 643 * dirac(k + 58) - 626 * dirac(k + 59) - 606 * dirac(k + 60) -
        583 * dirac(k + 61) - 557 * dirac(k + 62) - 528 * dirac(k + 63) - 496 * dirac(k + 64) -
        465 * dirac(k + 65) - 435 * dirac(k + 66) - 406 * dirac(k + 67) - 378 * dirac(k + 68) -
        351 * dirac(k + 69) - 325 * dirac(k + 70) - 300 * dirac(k + 71) - 276 * dirac(k + 72) -
        253 * dirac(k + 73) - 231 * dirac(k + 74) - 210 * dirac(k + 75) - 190 * dirac(k + 76) -
        171 * dirac(k + 77) - 153 * dirac(k + 78) - 136 * dirac(k + 79) - 120 * dirac(k + 80) -
        105 * dirac(k + 81) - 91 * dirac(k + 82) - 78 * dirac(k + 83) - 66 * dirac(k + 84) -
        55 * dirac(k + 85) - 45 * dirac(k + 86) - 36 * dirac(k + 87) - 28 * dirac(k + 88) -
        21 * dirac(k + 89) - 15 * dirac(k + 90) - 10 * dirac(k + 91) - 6 * dirac(k + 92) -
        3 * dirac(k + 93) - dirac(k + 94))

fig = go.Figure()
fig.add_trace(go.Bar(x=k_list, y=qj[6][0:len(k_list)], name=f'q{j}(k)'))
fig.update_layout(title=f'Scale {j}', xaxis_title='k', yaxis_title='q(k)', template='plotly_white')
fig.show()

k_list = []
j = 7
print('Scale =', j)
a = -(round(2**j) + round(2**(j-1)) - 2)  
b = -(1 - round(2**(j-1))) + 1            
for k in range(a, b):
    k_list.append(k)
    qj[7][k + abs(a)] = -1 / 8192 * (
        dirac(k-63) + 3*dirac(k-62) + 6*dirac(k-61) + 10*dirac(k-60) + 15*dirac(k-59) +
        21*dirac(k-58) + 28*dirac(k-57) + 36*dirac(k-56) + 45*dirac(k-55) +
        55*dirac(k-54) + 66*dirac(k-53) + 78*dirac(k-52) + 91*dirac(k-51) +
        105*dirac(k-50) + 120*dirac(k-49) + 136*dirac(k-48) + 153*dirac(k-47) +
        171*dirac(k-46) + 190*dirac(k-45) + 210*dirac(k-44) + 231*dirac(k-43) +
        253*dirac(k-42) + 276*dirac(k-41) + 300*dirac(k-40) + 325*dirac(k-39) +
        351*dirac(k-38) + 378*dirac(k-37) + 406*dirac(k-36) + 435*dirac(k-35) +
        465*dirac(k-34) + 496*dirac(k-33) + 528*dirac(k-32) + 561*dirac(k-31) +
        595*dirac(k-30) + 630*dirac(k-29) + 666*dirac(k-28) + 703*dirac(k-27) +
        741*dirac(k-26) + 780*dirac(k-25) + 820*dirac(k-24) + 861*dirac(k-23) +
        903*dirac(k-22) + 946*dirac(k-21) + 990*dirac(k-20) + 1035*dirac(k-19) +
        1081*dirac(k-18) + 1128*dirac(k-17) + 1176*dirac(k-16) + 1225*dirac(k-15) +
        1275*dirac(k-14) + 1326*dirac(k-13) + 1378*dirac(k-12) + 1431*dirac(k-11) +
        1485*dirac(k-10) + 1540*dirac(k-9) + 1596*dirac(k-8) + 1653*dirac(k-7) +
        1711*dirac(k-6) + 1770*dirac(k-5) + 1830*dirac(k-4) + 1891*dirac(k-3) +
        1953*dirac(k-2) + 2016*dirac(k-1) + 2080*dirac(k) + 2141*dirac(k+1) +
        2199*dirac(k+2) + 2254*dirac(k+3) + 2306*dirac(k+4) + 2355*dirac(k+5) +
        2401*dirac(k+6) + 2444*dirac(k+7) + 2484*dirac(k+8) + 2521*dirac(k+9) +
        2555*dirac(k+10) + 2586*dirac(k+11) + 2614*dirac(k+12) + 2639*dirac(k+13) +
        2661*dirac(k+14) + 2680*dirac(k+15) + 2696*dirac(k+16) + 2709*dirac(k+17) +
        2719*dirac(k+18) + 2726*dirac(k+19) + 2730*dirac(k+20) + 2731*dirac(k+21) +
        2729*dirac(k+22) + 2724*dirac(k+23) + 2716*dirac(k+24) + 2705*dirac(k+25) +
        2691*dirac(k+26) + 2674*dirac(k+27) + 2654*dirac(k+28) + 2631*dirac(k+29) +
        2605*dirac(k+30) + 2576*dirac(k+31) + 2544*dirac(k+32) + 2509*dirac(k+33) +
        2471*dirac(k+34) + 2430*dirac(k+35) + 2386*dirac(k+36) + 2339*dirac(k+37) +
        2289*dirac(k+38) + 2236*dirac(k+39) + 2180*dirac(k+40) + 2121*dirac(k+41) +
        2059*dirac(k+42) + 1994*dirac(k+43) + 1926*dirac(k+44) + 1855*dirac(k+45) +
        1781*dirac(k+46) + 1704*dirac(k+47) + 1624*dirac(k+48) + 1541*dirac(k+49) +
        1455*dirac(k+50) + 1366*dirac(k+51) + 1274*dirac(k+52) + 1179*dirac(k+53) +
        1081*dirac(k+54) + 980*dirac(k+55) + 876*dirac(k+56) + 769*dirac(k+57) +
        659*dirac(k+58) + 546*dirac(k+59) + 430*dirac(k+60) + 311*dirac(k+61) +
        189*dirac(k+62) + 64*dirac(k+63) - 64*dirac(k+64) - 189*dirac(k+65) -
        311*dirac(k+66) - 430*dirac(k+67) - 546*dirac(k+68) - 659*dirac(k+69) -
        769*dirac(k+70) - 876*dirac(k+71) - 980*dirac(k+72) - 1081*dirac(k+73) -
        1179*dirac(k+74) - 1274*dirac(k+75) - 1366*dirac(k+76) - 1455*dirac(k+77) -
        1541*dirac(k+78) - 1624*dirac(k+79) - 1704*dirac(k+80) - 1781*dirac(k+81) -
        1855*dirac(k+82) - 1926*dirac(k+83) - 1994*dirac(k+84) - 2059*dirac(k+85) -
        2121*dirac(k+86) - 2180*dirac(k+87) - 2236*dirac(k+88) - 2289*dirac(k+89) -
        2339*dirac(k+90) - 2386*dirac(k+91) - 2430*dirac(k+92) - 2471*dirac(k+93) -
        2509*dirac(k+94) - 2544*dirac(k+95) - 2576*dirac(k+96) - 2605*dirac(k+97) -
        2631*dirac(k+98) - 2654*dirac(k+99) - 2674*dirac(k+100) - 2691*dirac(k+101) -
        2705*dirac(k+102) - 2716*dirac(k+103) - 2724*dirac(k+104) - 2729*dirac(k+105) -
        2731*dirac(k+106) - 2730*dirac(k+107) - 2726*dirac(k+108) - 2719*dirac(k+109) -
        2709*dirac(k+110) - 2696*dirac(k+111) - 2680*dirac(k+112) - 2661*dirac(k+113) -
        2639*dirac(k+114) - 2614*dirac(k+115) - 2586*dirac(k+116) - 2555*dirac(k+117) -
        2521*dirac(k+118) - 2484*dirac(k+119) - 2444*dirac(k+120) - 2401*dirac(k+121) -
        2355*dirac(k+122) - 2306*dirac(k+123) - 2254*dirac(k+124) - 2199*dirac(k+125) -
        2141*dirac(k+126) - 2080*dirac(k+127) - 2016*dirac(k+128) - 1953*dirac(k+129) -
        1891*dirac(k+130) - 1830*dirac(k+131) - 1770*dirac(k+132) - 1711*dirac(k+133) -
        1653*dirac(k+134) - 1596*dirac(k+135) - 1540*dirac(k+136) - 1485*dirac(k+137) -
        1431*dirac(k+138) - 1378*dirac(k+139) - 1326*dirac(k+140) - 1275*dirac(k+141) -
        1225*dirac(k+142) - 1176*dirac(k+143) - 1128*dirac(k+144) - 1081*dirac(k+145) -
        1035*dirac(k+146) - 990*dirac(k+147) - 946*dirac(k+148) - 903*dirac(k+149) -
        861*dirac(k+150) - 820*dirac(k+151) - 780*dirac(k+152) - 741*dirac(k+153) -
        703*dirac(k+154) - 666*dirac(k+155) - 630*dirac(k+156) - 595*dirac(k+157) -
        561*dirac(k+158) - 528*dirac(k+159) - 496*dirac(k+160) - 465*dirac(k+161) -
        435*dirac(k+162) - 406*dirac(k+163) - 378*dirac(k+164) - 351*dirac(k+165) -
        325*dirac(k+166) - 300*dirac(k+167) - 276*dirac(k+168) - 253*dirac(k+169) -
        231*dirac(k+170) - 210*dirac(k+171) - 190*dirac(k+172) - 171*dirac(k+173) -
        153*dirac(k+174) - 136*dirac(k+175) - 120*dirac(k+176) - 105*dirac(k+177) -
        91*dirac(k+178) - 78*dirac(k+179) - 66*dirac(k+180) - 55*dirac(k+181) -
        45*dirac(k+182) - 36*dirac(k+183) - 28*dirac(k+184) - 21*dirac(k+185) -
        15*dirac(k+186) - 10*dirac(k+187) - 6*dirac(k+188) - 3*dirac(k+189) -
        dirac(k+190))

fig = go.Figure()
fig.add_trace(go.Bar(x=k_list, y=qj[7][0:len(k_list)], name=f'q{j}(k)'))
fig.update_layout(title=f'Scale {j}', xaxis_title='k', yaxis_title='q(k)', template='plotly_white')
fig.show()

k_list =[]
j = 8
print('Scale =', j)
a = -(round(2**j) + round(2**(j-1)) - 2)  
b = -(1 - round(2**(j-1))) + 1            
for k in range(a, b):
    k_list.append(k)
    eq1 = (dirac(k-127) + 3*dirac(k-126) + 6*dirac(k-125) + 10*dirac(k-124) + 15*dirac(k-123) +
            21*dirac(k-122) + 28*dirac(k-121) + 36*dirac(k-120) + 45*dirac(k-119) + 55*dirac(k-118) +
            66*dirac(k-117) + 78*dirac(k-116) + 91*dirac(k-115) + 105*dirac(k-114) + 120*dirac(k-113) +
            136*dirac(k-112) + 153*dirac(k-111) + 171*dirac(k-110) + 190*dirac(k-109) + 210*dirac(k-108) +
            231*dirac(k-107) + 253*dirac(k-106) + 276*dirac(k-105) + 300*dirac(k-104) + 325*dirac(k-103) +
            351*dirac(k-102) + 378*dirac(k-101) + 406*dirac(k-100) + 435*dirac(k-99) + 465*dirac(k-98) +
            496*dirac(k-97) + 528*dirac(k-96) + 561*dirac(k-95) + 595*dirac(k-94) + 630*dirac(k-93) +
            666*dirac(k-92) + 703*dirac(k-91) + 741*dirac(k-90) + 780*dirac(k-89) + 820*dirac(k-88) +
            861*dirac(k-87) + 903*dirac(k-86) + 946*dirac(k-85) + 990*dirac(k-84) + 1035*dirac(k-83) +
            1081*dirac(k-82) + 1128*dirac(k-81) + 1176*dirac(k-80) + 1225*dirac(k-79) + 1275*dirac(k-78) +
            1326*dirac(k-77) + 1378*dirac(k-76) + 1431*dirac(k-75) + 1485*dirac(k-74) + 1540*dirac(k-73) +
            1596*dirac(k-72) + 1653*dirac(k-71) + 1711*dirac(k-70) + 1770*dirac(k-69) + 1830*dirac(k-68) +
            1891*dirac(k-67) + 1953*dirac(k-66) + 2016*dirac(k-65) + 2080*dirac(k-64) + 2145*dirac(k-63) +
            2211*dirac(k-62) + 2278*dirac(k-61) + 2346*dirac(k-60) + 2415*dirac(k-59) + 2485*dirac(k-58) +
            2556*dirac(k-57) + 2628*dirac(k-56) + 2701*dirac(k-55) + 2775*dirac(k-54) + 2850*dirac(k-53) +
            2926*dirac(k-52) + 3003*dirac(k-51) + 3081*dirac(k-50) + 3160*dirac(k-49) + 3240*dirac(k-48) +
            3321*dirac(k-47) + 3403*dirac(k-46) + 3486*dirac(k-45) + 3570*dirac(k-44) + 3655*dirac(k-43) +
            3741*dirac(k-42) + 3828*dirac(k-41) + 3916*dirac(k-40) + 4005*dirac(k-39) + 4095*dirac(k-38) +
            4186*dirac(k-37) + 4278*dirac(k-36) + 4371*dirac(k-35) + 4465*dirac(k-34) + 4560*dirac(k-33) +
            4656*dirac(k-32) + 4753*dirac(k-31) + 4851*dirac(k-30) + 4950*dirac(k-29) + 5050*dirac(k-28) +
            5151*dirac(k-27) + 5253*dirac(k-26) + 5356*dirac(k-25) + 5460*dirac(k-24) + 5565*dirac(k-23) +
            5671*dirac(k-22) + 5778*dirac(k-21) + 5886*dirac(k-20) + 5995*dirac(k-19) + 6105*dirac(k-18) +
            6216*dirac(k-17) + 6328*dirac(k-16) + 6441*dirac(k-15) + 6555*dirac(k-14) + 6670*dirac(k-13) +
            6786*dirac(k-12) + 6903*dirac(k-11) + 7021*dirac(k-10) + 7140*dirac(k-9) + 7260*dirac(k-8) +
            7381*dirac(k-7) + 7503*dirac(k-6) + 7626*dirac(k-5) + 7750*dirac(k-4) + 7875*dirac(k-3) +
            8001*dirac(k-2) + 8128*dirac(k-1) + 8256*dirac(k) + 8381*dirac(k+1) + 8503*dirac(k+2) +
            8622*dirac(k+3) + 8738*dirac(k+4) + 8851*dirac(k+5) + 8961*dirac(k+6) + 9068*dirac(k+7) +
            9172*dirac(k+8) + 9273*dirac(k+9) + 9371*dirac(k+10) + 9466*dirac(k+11) + 9558*dirac(k+12) +
            9647*dirac(k+13) + 9733*dirac(k+14) + 9816*dirac(k+15) + 9896*dirac(k+16) + 9973*dirac(k+17) +
            10047*dirac(k+18) + 10118*dirac(k+19) + 10186*dirac(k+20) + 10251*dirac(k+21) + 10313*dirac(k+22) +
            10372*dirac(k+23) + 10428*dirac(k+24) + 10481*dirac(k+25) + 10531*dirac(k+26) + 10578*dirac(k+27) +
            10622*dirac(k+28) + 10663*dirac(k+29) + 10701*dirac(k+30) + 10736*dirac(k+31) + 10768*dirac(k+32) +
            10797*dirac(k+33) + 10823*dirac(k+34) + 10846*dirac(k+35) + 10866*dirac(k+36) + 10883*dirac(k+37) +
            10897*dirac(k+38) + 10908*dirac(k+39) + 10916*dirac(k+40) + 10921*dirac(k+41) + 10923*dirac(k+42) +
            10922*dirac(k+43) + 10918*dirac(k+44) + 10911*dirac(k+45) + 10901*dirac(k+46) + 10888*dirac(k+47) +
            10872*dirac(k+48) + 10853*dirac(k+49) + 10831*dirac(k+50) + 10806*dirac(k+51) + 10778*dirac(k+52) +
            10747*dirac(k+53) + 10713*dirac(k+54) + 10676*dirac(k+55) + 10636*dirac(k+56) + 10593*dirac(k+57) +
            10547*dirac(k+58) + 10498*dirac(k+59) + 10446*dirac(k+60) + 10391*dirac(k+61) + 10333*dirac(k+62) +
            10272*dirac(k+63) + 10208*dirac(k+64) + 10141*dirac(k+65) + 10071*dirac(k+66) + 9998*dirac(k+67) +
            9922*dirac(k+68) + 9843*dirac(k+69) + 9761*dirac(k+70) + 9676*dirac(k+71) + 9588*dirac(k+72) +
            9497*dirac(k+73) + 9403*dirac(k+74) + 9306*dirac(k+75) + 9206*dirac(k+76) + 9103*dirac(k+77) +
            8997*dirac(k+78) + 8888*dirac(k+79) + 8776*dirac(k+80) + 8661*dirac(k+81) + 8543*dirac(k+82) +
            8422*dirac(k+83) + 8298*dirac(k+84) + 8171*dirac(k+85) + 8041*dirac(k+86) + 7908*dirac(k+87) +
            7772*dirac(k+88) + 7633*dirac(k+89) + 7491*dirac(k+90) + 7346*dirac(k+91) + 7198*dirac(k+92) +
            7047*dirac(k+93) + 6893*dirac(k+94) + 6736*dirac(k+95) + 6576*dirac(k+96) + 6413*dirac(k+97) +
            6247*dirac(k+98) + 6078*dirac(k+99) + 5906*dirac(k+100) + 5731*dirac(k+101) + 5553*dirac(k+102) +
            5372*dirac(k+103) + 5188*dirac(k+104) + 5001*dirac(k+105) + 4811*dirac(k+106) + 4618*dirac(k+107) +
            4422*dirac(k+108) + 4223*dirac(k+109) + 4021*dirac(k+110) + 3816*dirac(k+111) + 3608*dirac(k+112) +
            3397*dirac(k+113) + 3183*dirac(k+114) + 2966*dirac(k+115) + 2746*dirac(k+116) + 2523*dirac(k+117) +
            2297*dirac(k+118) + 2068*dirac(k+119) + 1836*dirac(k+120) + 1601*dirac(k+121) + 1363*dirac(k+122) +
            1122*dirac(k+123) + 878*dirac(k+124) + 631*dirac(k+125) + 381*dirac(k+126) + 128*dirac(k+127))
    eq2 = (- 128*dirac(k+128) - 381*dirac(k+129) - 631*dirac(k+130) - 878*dirac(k+131) - 1122*dirac(k+132) -
            1363*dirac(k+133) - 1601*dirac(k+134) - 1836*dirac(k+135) - 2068*dirac(k+136) - 2297*dirac(k+137) -
            2523*dirac(k+138) - 2746*dirac(k+139) - 2966*dirac(k+140) - 3183*dirac(k+141) - 3397*dirac(k+142) -
            3608*dirac(k+143) - 3816*dirac(k+144) - 4021*dirac(k+145) - 4223*dirac(k+146) - 4422*dirac(k+147) -
            4618*dirac(k+148) - 4811*dirac(k+149) - 5001*dirac(k+150) - 5188*dirac(k+151) - 5372*dirac(k+152) -
            5553*dirac(k+153) - 5731*dirac(k+154) - 5906*dirac(k+155) - 6078*dirac(k+156) - 6247*dirac(k+157) -
            6413*dirac(k+158) - 6576*dirac(k+159) - 6736*dirac(k+160) - 6893*dirac(k+161) - 7047*dirac(k+162) -
            7198*dirac(k+163) - 7346*dirac(k+164) - 7491*dirac(k+165) - 7633*dirac(k+166) - 7772*dirac(k+167) -
            7908*dirac(k+168) - 8041*dirac(k+169) - 8171*dirac(k+170) - 8298*dirac(k+171) - 8422*dirac(k+172) -
            8543*dirac(k+173) - 8661*dirac(k+174) - 8776*dirac(k+175) - 8888*dirac(k+176) - 8997*dirac(k+177) - 
            9103*dirac(k+178) - 9206*dirac(k+179) -
            9306*dirac(k+180) - 9403*dirac(k+181) - 9497*dirac(k+182) - 9588*dirac(k+183) - 9676*dirac(k+184) -
            9761*dirac(k+185) - 9843*dirac(k+186) - 9922*dirac(k+187) - 9998*dirac(k+188) - 10071*dirac(k+189) -
            10141*dirac(k+190) - 10208*dirac(k+191) - 10272*dirac(k+192) - 10333*dirac(k+193) - 10391*dirac(k+194) -
            10446*dirac(k+195) - 10498*dirac(k+196) - 10547*dirac(k+197) - 10593*dirac(k+198) - 10636*dirac(k+199) -
            10676*dirac(k+200) - 10713*dirac(k+201) - 10747*dirac(k+202) - 10778*dirac(k+203) - 10806*dirac(k+204) -
            10831*dirac(k+205) - 10853*dirac(k+206) - 10872*dirac(k+207) - 10888*dirac(k+208) - 10901*dirac(k+209) -
            10911*dirac(k+210) - 10918*dirac(k+211) - 10922*dirac(k+212) - 10923*dirac(k+213) - 10921*dirac(k+214) -
            10916*dirac(k+215) - 10908*dirac(k+216) - 10897*dirac(k+217) - 10883*dirac(k+218) - 10866*dirac(k+219) -
            10846*dirac(k+220) - 10823*dirac(k+221) - 10797*dirac(k+222) - 10768*dirac(k+223) - 10736*dirac(k+224) -
            10701*dirac(k+225) - 10663*dirac(k+226) - 10622*dirac(k+227) - 10578*dirac(k+228) - 10531*dirac(k+229) -
            10481*dirac(k+230) - 10428*dirac(k+231) - 10372*dirac(k+232) - 10313*dirac(k+233) - 10251*dirac(k+234) -
            10186*dirac(k+235) - 10118*dirac(k+236) - 10047*dirac(k+237) - 9973*dirac(k+238) - 9896*dirac(k+239) -
            9816*dirac(k+240) - 9733*dirac(k+241) - 9647*dirac(k+242) - 9558*dirac(k+243) - 9466*dirac(k+244) -
            9371*dirac(k+245) - 9273*dirac(k+246) - 9172*dirac(k+247) - 9068*dirac(k+248) - 8961*dirac(k+249) -
            8851*dirac(k+250) - 8738*dirac(k+251) - 8622*dirac(k+252) - 8503*dirac(k+253) - 8381*dirac(k+254) -
            8256*dirac(k+255) - 8128*dirac(k+256) - 8001*dirac(k+257) - 7875*dirac(k+258) - 7750*dirac(k+259) -
            7626*dirac(k+260) - 7503*dirac(k+261) - 7381*dirac(k+262) - 7260*dirac(k+263) - 7140*dirac(k+264) -
            7021*dirac(k+265) - 6903*dirac(k+266) - 6786*dirac(k+267) - 6670*dirac(k+268) - 6555*dirac(k+269) -
            6441*dirac(k+270) - 6328*dirac(k+271) - 6216*dirac(k+272) - 6105*dirac(k+273) - 5995*dirac(k+274) -
            5886*dirac(k+275) - 5778*dirac(k+276) - 5671*dirac(k+277) - 5565*dirac(k+278) - 5460*dirac(k+279) -
            5356*dirac(k+280) - 5253*dirac(k+281) - 5151*dirac(k+282) - 5050*dirac(k+283) - 4950*dirac(k+284) -
            4851*dirac(k+285) - 4753*dirac(k+286) - 4656*dirac(k+287) - 4560*dirac(k+288) - 4465*dirac(k+289) -
            4371*dirac(k+290) - 4278*dirac(k+291) - 4186*dirac(k+292) - 4095*dirac(k+293) - 4005*dirac(k+294) -
            3916*dirac(k+295) - 3828*dirac(k+296) - 3741*dirac(k+297) - 3655*dirac(k+298) - 3570*dirac(k+299) -
            3486*dirac(k+300) - 3403*dirac(k+301) - 3321*dirac(k+302) - 3240*dirac(k+303) - 3160*dirac(k+304) -
            3081*dirac(k+305) - 3003*dirac(k+306) - 2926*dirac(k+307) - 2850*dirac(k+308) - 2775*dirac(k+309) -
            2701*dirac(k+310) - 2628*dirac(k+311) - 2556*dirac(k+312) - 2485*dirac(k+313) - 2415*dirac(k+314) -
            2346*dirac(k+315) - 2278*dirac(k+316) - 2211*dirac(k+317) - 2145*dirac(k+318) - 2080*dirac(k+319) -
            2016*dirac(k+320) - 1953*dirac(k+321) - 1891*dirac(k+322) - 1830*dirac(k+323) - 1770*dirac(k+324) -
            1711*dirac(k+325) - 1653*dirac(k+326) - 1596*dirac(k+327) - 1540*dirac(k+328) - 1485*dirac(k+329) -
            1431*dirac(k+330) - 1378*dirac(k+331) - 1326*dirac(k+332) - 1275*dirac(k+333) - 1225*dirac(k+334) -
            1176*dirac(k+335) - 1128*dirac(k+336) - 1081*dirac(k+337) - 1035*dirac(k+338) - 990*dirac(k+339) -
            946*dirac(k+340) - 903*dirac(k+341) - 861*dirac(k+342) - 820*dirac(k+343) - 780*dirac(k+344) -
            741*dirac(k+345) - 703*dirac(k+346) - 666*dirac(k+347) - 630*dirac(k+348) - 595*dirac(k+349) -
            561*dirac(k+350) - 528*dirac(k+351) - 496*dirac(k+352) - 465*dirac(k+353) - 435*dirac(k+354) -
            406*dirac(k+355) - 378*dirac(k+356) - 351*dirac(k+357) - 325*dirac(k+358) - 300*dirac(k+359) -
            276*dirac(k+360) - 253*dirac(k+361) - 231*dirac(k+362) - 210*dirac(k+363) - 190*dirac(k+364) -
            171*dirac(k+365) - 153*dirac(k+366) - 136*dirac(k+367) - 120*dirac(k+368) - 105*dirac(k+369) -
            91*dirac(k+370) - 78*dirac(k+371) - 66*dirac(k+372) - 55*dirac(k+373) - 45*dirac(k+374) -
            36*dirac(k+375) - 28*dirac(k+376) - 21*dirac(k+377) - 15*dirac(k+378) - 10*dirac(k+379) -
            6*dirac(k+380) - 3*dirac(k+381) - dirac(k+382))
    qj[8][k + abs(a)] = (-1 / 32768*(eq1+eq2))

fig = go.Figure()
fig.add_trace(go.Bar(x=k_list, y=qj[8][0:len(k_list)], name=f'q{j}(k)'))
fig.update_layout(title=f'Scale {j}', xaxis_title='k', yaxis_title='q(k)', template='plotly_white')
fig.show()

T1 = round(2**(1-1)) - 1
T2 = round(2**(2-1)) - 1
T3 = round(2**(3-1)) - 1
T4 = round(2**(4-1)) - 1
T5 = round(2**(5-1)) - 1
T6 = round(2**(6-1)) - 1
T7 = round(2**(7-1)) - 1
T8 = round(2**(8-1)) - 1

print('T1 =', T1)
print('T2 =', T2)
print('T3 =', T3)
print('T4 =', T4)
print('T5 =', T5)
print('T6 =', T6)
print('T7 =', T7)
print('T8 =', T8)


h(n)


g(n)


Scale = 1
a = -1
b = 1


Scale = 2
a = -4
b = 2


Scale = 3
a = -10
b = 4


...

Scale = 5
a = -46
b = 16


Scale = 6


Scale = 7


Scale = 8


T1 = 0
T2 = 1
T3 = 3
T4 = 7
T5 = 15
T6 = 31
T7 = 63
T8 = 127


## Frequency Response Analysis

In [51]:
# Get the sampling rate and nyquist frequency
fs = target_sampling_rate  # 50 Hz
nyquist_freq = fs / 2  # 25 Hz

# Calculate frequency response for each scale
scale_frequencies = {}
frequency_ranges = {}

print("Wavelet Scale Frequency Analysis:")
print("=" * 50)

for j in range(1, 9):
    # The center frequency for each scale is approximately fs/(2^(j+1))
    center_freq = fs / (2**(j+1))
    
    # The frequency range for each scale (approximate)
    # Lower bound: fs/(2^(j+2))
    # Upper bound: fs/(2^j)
    freq_low = fs / (2**(j+2)) if j < 8 else 0
    freq_high = fs / (2**j)
    
    scale_frequencies[j] = center_freq
    frequency_ranges[j] = (freq_low, freq_high)
    
    print(f"Scale {j}:")
    print(f"  Center Frequency: {center_freq:.3f} Hz")
    print(f"  Frequency Range: {freq_low:.3f} - {freq_high:.3f} Hz")
    print(f"  Period: {1/center_freq:.3f} seconds")
    print()

# Create a visualization of frequency bands
fig = go.Figure()

colors = ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'violet', 'pink']

for j in range(1, 9):
    freq_low, freq_high = frequency_ranges[j]
    center_freq = scale_frequencies[j]
    
    # Add frequency band as a rectangle
    fig.add_shape(
        type="rect",
        x0=freq_low, x1=freq_high,
        y0=j-0.4, y1=j+0.4,
        fillcolor=colors[j-1],
        opacity=0.6,
        line=dict(color=colors[j-1], width=2)
    )
    
    # Add center frequency marker
    fig.add_trace(go.Scatter(
        x=[center_freq],
        y=[j],
        mode='markers+text',
        marker=dict(color='black', size=8),
        text=[f'{center_freq:.2f} Hz'],
        textposition='middle right',
        showlegend=False,
        name=f'Scale {j} Center'
    ))

fig.update_layout(
    title='Frequency Response of Each Wavelet Scale',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Wavelet Scale',
    template='plotly_white',
    height=500,
    xaxis=dict(range=[0, nyquist_freq]),
    yaxis=dict(
        tickvals=list(range(1, 9)),
        ticktext=[f'Scale {i}' for i in range(1, 9)]
    )
)

fig.show()

# Summary table
import pandas as pd

frequency_table = pd.DataFrame({
    'Scale': list(range(1, 9)),
    'Center_Frequency_Hz': [scale_frequencies[j] for j in range(1, 9)],
    'Freq_Range_Low_Hz': [frequency_ranges[j][0] for j in range(1, 9)],
    'Freq_Range_High_Hz': [frequency_ranges[j][1] for j in range(1, 9)],
    'Period_seconds': [1/scale_frequencies[j] for j in range(1, 9)]
})

print("\nFrequency Response Summary Table:")
print(frequency_table.to_string(index=False, float_format='%.3f'))

Wavelet Scale Frequency Analysis:
Scale 1:
  Center Frequency: 12.500 Hz
  Frequency Range: 6.250 - 25.000 Hz
  Period: 0.080 seconds

Scale 2:
  Center Frequency: 6.250 Hz
  Frequency Range: 3.125 - 12.500 Hz
  Period: 0.160 seconds

Scale 3:
  Center Frequency: 3.125 Hz
  Frequency Range: 1.562 - 6.250 Hz
  Period: 0.320 seconds

Scale 4:
  Center Frequency: 1.562 Hz
  Frequency Range: 0.781 - 3.125 Hz
  Period: 0.640 seconds

Scale 5:
  Center Frequency: 0.781 Hz
  Frequency Range: 0.391 - 1.562 Hz
  Period: 1.280 seconds

Scale 6:
  Center Frequency: 0.391 Hz
  Frequency Range: 0.195 - 0.781 Hz
  Period: 2.560 seconds

Scale 7:
  Center Frequency: 0.195 Hz
  Frequency Range: 0.098 - 0.391 Hz
  Period: 5.120 seconds

Scale 8:
  Center Frequency: 0.098 Hz
  Frequency Range: 0.000 - 0.195 Hz
  Period: 10.240 seconds




Frequency Response Summary Table:
 Scale  Center_Frequency_Hz  Freq_Range_Low_Hz  Freq_Range_High_Hz  Period_seconds
     1               12.500              6.250              25.000           0.080
     2                6.250              3.125              12.500           0.160
     3                3.125              1.562               6.250           0.320
     4                1.562              0.781               3.125           0.640
     5                0.781              0.391               1.562           1.280
     6                0.391              0.195               0.781           2.560
     7                0.195              0.098               0.391           5.120
     8                0.098              0.000               0.195          10.240


In [52]:
# Delal Table
delay_data = {
    "Scale (j)": [],
    "Delay T (sample)": [],
    "Delay in seconds (T / fs)": []
}

for j in range(1, 9):
    T = round(2**(j-1)) - 1
    delay_data["Scale (j)"].append(j)
    delay_data["Delay T (sample)"].append(T)
    delay_data["Delay in seconds (T / fs)"].append(T / fs)

df_delay = pd.DataFrame(delay_data)
print("Delay Table:")
display(df_delay)

# FREQ DOMAIN 8 SCALE
h = []
g = []
n_list = []

for n in range(-2, 2):
    n_list.append(n)
    temp_h = 1/8 * (dirac(n-1) + 3*dirac(n) + 3*dirac(n+1) + dirac(n+2))
    temp_g = -2 * (dirac(n) - dirac(n+1))
    h.append(temp_h)
    g.append(temp_g)

# Domain freq
Hw = np.zeros(fs+1)
Gw = np.zeros(fs+1)
i_list = []

for i in range(fs+1):
    i_list.append(i)
    reG = imG = reH = imH = 0
    for k in range(-2, 2):
        idx = k + 2
        reG += g[idx] * np.cos(k * 2 * np.pi * i / fs)
        imG -= g[idx] * np.sin(k * 2 * np.pi * i / fs)
        reH += h[idx] * np.cos(k * 2 * np.pi * i / fs)
        imH -= h[idx] * np.sin(k * 2 * np.pi * i / fs)

    Hw[i] = np.sqrt(reH**2 + imH**2)
    Gw[i] = np.sqrt(reG**2 + imG**2)

i_list = i_list[:round(fs/2)+1]

Q = np.zeros((9, round(fs/2)+1))
for i in range(round(fs/2)+1):
    Q[1][i] = Gw[i]
    Q[2][i] = Gw[2*i]*Hw[i] if 2*i < len(Gw) else 0
    Q[3][i] = Gw[4*i]*Hw[2*i]*Hw[i] if 4*i < len(Gw) else 0
    Q[4][i] = Gw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i] if 8*i < len(Gw) else 0
    Q[5][i] = Gw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i] if 16*i < len(Gw) else 0
    Q[6][i] = Gw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i] if 32*i < len(Gw) else 0
    Q[7][i] = Gw[64*i]*Hw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i] if 64*i < len(Gw) else 0
    Q[8][i] = Gw[128*i]*Hw[64*i]*Hw[32*i]*Hw[16*i]*Hw[8*i]*Hw[4*i]*Hw[2*i]*Hw[i] if 128*i < len(Gw) else 0

print("\nFrequency Response 8 Scale:")
fig = go.Figure()
for level in range(1, 9):
    fig.add_trace(go.Scatter(
        x=i_list,
        y=Q[level],
        mode='lines',
        name=f'Q{level}',
        line=dict(width=2),
    ))

fig.update_layout(
    title='Frequency Response of 8 Wavelet Scales',
    xaxis_title='Frequency (Hz)',
    yaxis_title='Magnitude',
    template='plotly_white',
    height=500,
    legend=dict(x=0.7, y=0.95),
    showlegend=True,
)
fig.show()

data = []
for j in range(1, 9):  # Skala 1 sampai 8
    f_min = fs / (2**(j + 2))
    f_max = fs / (2**j)
    bandwidth = f_max - f_min
    data.append({
        "Scale": j,
        "Minimum Frequency (Hz)": f_min,
        "Maximum Frequency (Hz)": f_max,
        "Bandwidth (Hz)": bandwidth
    })

df_freq = pd.DataFrame(data)

def highlight_scale_8(row):
    return ['background-color: lightblue' if row.Scale == 8 else '' for _ in row]

print("\nFrequency Range and Bandwidth of Each DWT Scale:")
display(df_freq)

Delay Table:


,Scale (j),Delay T (sample),Delay in seconds (T / fs)
0,1,0,0.00
1,2,1,0.02
2,3,3,0.06
3,4,7,0.14
4,5,15,0.30
5,6,31,0.62
6,7,63,1.26
7,8,127,2.54



Frequency Response 8 Scale:



Frequency Range and Bandwidth of Each DWT Scale:


,Scale,Minimum Frequency (Hz),Maximum Frequency (Hz),Bandwidth (Hz)
0,1,6.250000,25.000000,18.750000
1,2,3.125000,12.500000,9.375000
2,3,1.562500,6.250000,4.687500
3,4,0.781250,3.125000,2.343750
4,5,0.390625,1.562500,1.171875
5,6,0.195312,0.781250,0.585938
6,7,0.097656,0.390625,0.292969
7,8,0.048828,0.195312,0.146484


## Implement DWT

In [53]:
w2fb = np.zeros((9, 100000)) 
n_list = []
for n in range(0, (total_time * target_sampling_rate)):
    n_list.append(n)
    
    for j in range(1, 9):
        w2fb[1][n+T1] = 0
        w2fb[2][n+T2] = 0
        w2fb[3][n+T3] = 0
        w2fb[4][n+T4] = 0
        w2fb[5][n+T5] = 0
        w2fb[6][n+T6] = 0
        w2fb[7][n+T7] = 0
        w2fb[8][n+T8] = 0

        a = -(round(2**j) + round(2**(j-1)) - 2)
        b = -(a - round(2**(j-1)))

        for k in range(a, b+1):
            w2fb[1][n+T1] = w2fb[1][n+T1] + qj[1][k + abs(a)] * signal_ds[n - (k + abs(a))]
            w2fb[2][n+T2] = w2fb[2][n+T2] + qj[2][k + abs(a)] * signal_ds[n - (k + abs(a))]
            w2fb[3][n+T3] = w2fb[3][n+T3] + qj[3][k + abs(a)] * signal_ds[n - (k + abs(a))]
            w2fb[4][n+T4] = w2fb[4][n+T4] + qj[4][k + abs(a)] * signal_ds[n - (k + abs(a))]
            w2fb[5][n+T5] = w2fb[5][n+T5] + qj[5][k + abs(a)] * signal_ds[n - (k + abs(a))]
            w2fb[6][n+T6] = w2fb[6][n+T6] + qj[6][k + abs(a)] * signal_ds[n - (k + abs(a))]
            w2fb[7][n+T7] = w2fb[7][n+T7] + qj[7][k + abs(a)] * signal_ds[n - (k + abs(a))]
            w2fb[8][n+T8] = w2fb[8][n+T8] + qj[8][k + abs(a)] * signal_ds[n - (k + abs(a))]

for i in range(1, 9):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=time,
        y=w2fb[i][0:len(n_list)],
        mode='lines',
        name=f'Orde {i}'
    ))
    
    fig.update_layout(
        title=f'Plot Orde {i}',
        xaxis_title='Waktu (detik)',
        yaxis_title=f'Amplitudo',
        template='plotly_white'
    )
    
    fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_ds, y=signal_ds, mode='lines', name='PPG Signal (Downsampled)', line=dict(color='royalblue')))
fig.update_layout(title='PPG Signal (Downsampled to 50 Hz)',
                  xaxis_title='Time (seconds)',
                  yaxis_title='Amplitude',
                  template='plotly_white',
                  font=dict(size=14),
                  margin=dict(l=40, r=40, t=60, b=40))
fig.show()

## Respiratory Rate Extraction

In [54]:
from scipy.signal import find_peaks
import numpy as np

def detect_peaks(signal, sampling_rate=50):
    # Prominence-based detection
    peaks, properties = find_peaks(
        signal,
        prominence=np.std(signal) * 0.3,
        distance=int(0.4 * sampling_rate)
    )
    
    return peaks, properties

# Apply peak detection to w2fb[7]
print("Peak Detection Analysis for Wavelet Scale 7")
print("=" * 50)

# Extract the signal from scale 7 (assuming w2fb is available)
# Get the signal length that matches the original data
signal_length = len(signal_ds)
scale_7_signal = w2fb[7][:signal_length]

# Create time array
time_array = np.arange(len(scale_7_signal)) / target_sampling_rate

detected_peaks = {}
peaks, props = detect_peaks(scale_7_signal, target_sampling_rate)
    
print(f"  Number of peaks detected: {len(peaks)}")
if len(peaks) > 0:
    peak_times = time_array[peaks]
    peak_values = scale_7_signal[peaks]
    print(f"  Peak times (seconds): {peak_times[:5]}{'...' if len(peak_times) > 5 else ''}")
    print(f"  Peak amplitudes: {peak_values[:5]}{'...' if len(peak_values) > 5 else ''}")

# Visualization of Peak Detection Results
fig = go.Figure()

# Plot the original signal
fig.add_trace(go.Scatter(
    x=time_array,
    y=scale_7_signal,
    mode='lines',
    name='Scale 7 Signal',
    line=dict(color='blue', width=1),
    opacity=0.7
))

if len(peaks) > 0:
    fig.add_trace(go.Scatter(
        x=time_array[peaks],
        y=scale_7_signal[peaks],
        mode='markers',
        name=f'Detected Peaks ({len(peaks)})',
        marker=dict(
            color='green',
            size=8,
            symbol='circle',
            line=dict(width=2, color='white')
        )
    ))

fig_detailed = go.Figure()
fig_detailed.add_trace(go.Scatter(
    x=time_array,
    y=scale_7_signal,
    mode='lines',
    name='Scale 7 Signal',
    line=dict(color='blue', width=1.5)
))

# Detected peaks
if len(peaks) > 0:
    fig_detailed.add_trace(go.Scatter(
        x=time_array[peaks],
        y=scale_7_signal[peaks],
        mode='markers',
        name=f'Detected Peaks ({len(peaks)})',
        marker=dict(
            color='green',
            size=10,
            symbol='circle',
            line=dict(width=2, color='white')
        )
    ))
    
    # Add vertical lines at peak locations (first 15 peaks to avoid clutter)
    for i, peak_idx in enumerate(peaks[:15]):
        fig_detailed.add_vline(
            x=time_array[peak_idx],
            line_dash="dash",
            line_color="green",
            opacity=0.4,
            annotation_text=f"P{i+1}" if i < 10 else "",
            annotation_position="top"
        )

fig_detailed.update_layout(
    title='Peak Detection Results',
    xaxis_title='Time (seconds)',
    yaxis_title='Amplitude',
    template='plotly_white',
    height=500,
    showlegend=True
)

fig_detailed.show()

Peak Detection Analysis for Wavelet Scale 7
  Number of peaks detected: 14
  Peak times (seconds): [ 1.98  4.92  8.56 13.6  24.28]...
  Peak amplitudes: [  236.33776308 -1371.10049562  2930.11362587  1284.63452226
  2598.92521416]...


## Vasometric Activity Extraction

In [55]:
# Extract Scale 8 signal
print("Scale 8 Vasometric Activity Analysis")
print("=" * 50)

# Get scale 8 data
signal_length = len(signal_ds)
scale_8_signal = w2fb[8][:signal_length]
time_array = np.arange(len(scale_8_signal)) / target_sampling_rate

print(f"Signal length: {len(scale_8_signal)} samples")
print(f"Duration: {time_array[-1]:.2f} seconds")
print(f"Sampling rate: {target_sampling_rate} Hz")

# Plot Scale 8 signal
fig_scale8 = go.Figure()
fig_scale8.add_trace(go.Scatter(
    x=time_array,
    y=scale_8_signal,
    mode='lines',
    name='Scale 8 Signal',
    line=dict(color='blue', width=1.5)
))

fig_scale8.update_layout(
    title='Scale 8 Signal (Vasometric Activity)',
    xaxis_title='Time (seconds)',
    yaxis_title='Amplitude',
    template='plotly_white',
    height=400
)
fig_scale8.show()

Scale 8 Vasometric Activity Analysis
Signal length: 3000 samples
Duration: 59.98 seconds
Sampling rate: 50 Hz


In [56]:
# Peak Frequency Detection and Extraction

def find_spectral_peaks(frequencies, magnitudes, min_prominence=None, num_peaks=5):
    """
    Find peaks in the frequency spectrum
    """
    if min_prominence is None:
        min_prominence = np.max(magnitudes) * 0.1  # 10% of max magnitude
    
    peaks = []
    peak_magnitudes = []
    
    for i in range(1, len(magnitudes)-1):
        # Check if current point is higher than neighbors and above prominence
        if (magnitudes[i] > magnitudes[i-1] and 
            magnitudes[i] > magnitudes[i+1] and 
            magnitudes[i] > min_prominence):
            peaks.append(i)
            peak_magnitudes.append(magnitudes[i])
    
    # Sort peaks by magnitude (descending)
    if peaks:
        peak_indices = np.array(peaks)
        peak_mags = np.array(peak_magnitudes)
        sorted_indices = np.argsort(peak_mags)[::-1]
        
        # Return top N peaks
        top_peaks = peak_indices[sorted_indices[:num_peaks]]
        top_magnitudes = peak_mags[sorted_indices[:num_peaks]]
        
        return top_peaks, top_magnitudes
    else:
        return [], []

# Find peak frequencies
print("Peak Frequency Detection")
print("=" * 30)

max_freq_idx = len(freq_positive) // 2
freq_analysis = freq_positive[:max_freq_idx]
mag_analysis = magnitude_positive[:max_freq_idx]

peak_indices, peak_magnitudes = find_spectral_peaks(freq_analysis, mag_analysis, num_peaks=10)

if len(peak_indices) > 0:
    peak_frequencies = freq_analysis[peak_indices]
    
    print(f"Found {len(peak_indices)} significant peaks:")
    print("-" * 50)
    
    peak_data = []
    for i, (freq, mag, idx) in enumerate(zip(peak_frequencies, peak_magnitudes, peak_indices)):
        # Calculate relative power
        total_power = np.sum(mag_analysis**2)
        peak_power = mag**2
        relative_power = (peak_power / total_power) * 100
        
        peak_info = {
            'Rank': i+1,
            'Frequency_Hz': round(freq, 4),
            'Magnitude': round(mag, 6),
            'Power': round(peak_power, 6),
            'Relative_Power_%': round(relative_power, 2)
        }
        peak_data.append(peak_info)
        
        print(f"Peak {i+1}:")
        print(f"  Frequency: {freq:.4f} Hz")
        print(f"  Magnitude: {mag:.6f}")
        print(f"  Power: {peak_power:.6f}")
        print(f"  Relative Power: {relative_power:.2f}%")
    
    # Create DataFrame for better visualization
    df_peaks = pd.DataFrame(peak_data)
    print("Peak Frequency Summary Table:")
    display(df_peaks)
    
    # Visualize detected peaks on spectrum
    fig_peaks = go.Figure()
    
    # Plot full spectrum
    fig_peaks.add_trace(go.Scatter(
        x=freq_analysis,
        y=mag_analysis,
        mode='lines',
        name='Magnitude Spectrum',
        line=dict(color='blue', width=1)
    ))
    
    # Plot detected peaks
    fig_peaks.add_trace(go.Scatter(
        x=peak_frequencies,
        y=peak_magnitudes,
        mode='markers',
        name=f'Detected Peaks ({len(peak_frequencies)})',
        marker=dict(
            color='red',
            size=10,
            symbol='circle',
            line=dict(width=2, color='white')
        )
    ))
    
    # Add annotations for top 3 peaks
    for i, (freq, mag) in enumerate(zip(peak_frequencies[:3], peak_magnitudes[:3])):
        fig_peaks.add_annotation(
            x=freq,
            y=mag,
            text=f'Peak {i+1}<br>{freq:.4f} Hz',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor='red',
            bgcolor='white',
            bordercolor='red',
            borderwidth=1
        )
    
    fig_peaks.update_layout(
        title='Scale 8 - Peak Frequency Detection Results',
        xaxis_title='Frequency (Hz)',
        yaxis_title='Magnitude',
        template='plotly_white',
        height=500,
        legend=dict(x=0.7, y=0.95)
    )
    fig_peaks.show()
    
    # Statistical summary
    print("\nStatistical Summary:")
    print("-" * 20)
    dominant_freq = peak_frequencies[0]
    print(f"Dominant frequency: {dominant_freq:.4f} Hz")
    print(f"Dominant period: {1/dominant_freq:.2f} seconds")
    print(f"Total spectral peaks: {len(peak_indices)}")
    print(f"Frequency range: {freq_analysis[0]:.4f} - {freq_analysis[-1]:.4f} Hz")
    print(f"Spectral resolution: {freq_analysis[1] - freq_analysis[0]:.6f} Hz")
else:
    print("No significant peaks detected in the spectrum.")
    print("This could indicate:")
    print("- Very low signal energy")
    print("- Uniform noise distribution") 
    print("- Need to adjust prominence threshold")

Peak Frequency Detection
Found 5 significant peaks:
--------------------------------------------------
Peak 1:
  Frequency: 0.0854 Hz
  Magnitude: 3723794.076144
  Power: 13866642321528.046875
  Relative Power: 20.76%
Peak 2:
  Frequency: 0.0244 Hz
  Magnitude: 3122871.834762
  Power: 9752328496349.056641
  Relative Power: 14.60%
Peak 3:
  Frequency: 0.1343 Hz
  Magnitude: 1865403.689221
  Power: 3479730923758.556641
  Relative Power: 5.21%
Peak 4:
  Frequency: 0.1099 Hz
  Magnitude: 1244948.977867
  Power: 1549897957491.051025
  Relative Power: 2.32%
Peak 5:
  Frequency: 0.1831 Hz
  Magnitude: 809192.740418
  Power: 654792891145.947021
  Relative Power: 0.98%
Peak Frequency Summary Table:


,Rank,Frequency_Hz,Magnitude,Power,Relative_Power_%
0,1,0.0854,3.723794e+06,1.386664e+13,20.76
1,2,0.0244,3.122872e+06,9.752328e+12,14.60
2,3,0.1343,1.865404e+06,3.479731e+12,5.21
3,4,0.1099,1.244949e+06,1.549898e+12,2.32
4,5,0.1831,8.091927e+05,6.547929e+11,0.98



Statistical Summary:
--------------------
Dominant frequency: 0.0854 Hz
Dominant period: 11.70 seconds
Total spectral peaks: 5
Frequency range: 0.0000 - 12.4878 Hz
Spectral resolution: 0.012207 Hz


---